<a href="https://colab.research.google.com/github/Adamkeske/DataSciencePortfolio/blob/main/ADAM_KESKE_Final_Project_for_BIA_681.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prediction of Home Sale Price**

Adam Keske


1.   Background
2.   Goal
1.   Source Data
2.   Initial Review of the Data
2.   Data Wrangling (if necessary).
6.   Machine Learning Methodologies (Only if you are using a structured dataset)
7.   Deep Learning Methodologies
8.   Overall Conclusions and Recommendations


**Background**

This data set encompasses many different factors in order to help determine house prices across the country.







**Goal**

Goal- The goal of this research is to be able to accuratley predict home prices, by doing this it allows the end user to predict and estimate home prices so the user can pay the correct amount for the home.

**Source Data**

Data was collected from kaggle datasets. Data head will be shown below.

In [1]:
import pandas as pd
import numpy as np
import os 
import sys
import scipy
import sklearn
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model           import LinearRegression
from sklearn.linear_model           import LogisticRegression
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from scipy import stats

%matplotlib inline




**Intial Data Review**

In [2]:


from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/train.csv"
df1= pd.read_csv('/content/drive/MyDrive/train.csv')
df1

Mounted at /content/drive


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,...,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,...,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,...,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,G

**Data Wrangling**

In [3]:
df1.drop(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Condition1', 'Condition2', 'HouseStyle','RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'HeatingQC','LowQualFinSF', 'GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageArea', 'GarageQual',
       'GarageCond', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeature', 'MiscVal'], axis=1, inplace=True)
df1.columns

Index(['Neighborhood', 'BldgType', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'Heating', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'Fireplaces', 'GarageCars', 'PavedDrive', 'MoSold', 'YrSold',
       'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [4]:
df1.drop(['YearRemodAdd', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtFullBath', 'BsmtHalfBath'], axis=1, inplace=True)

In [ ]:
sum_column = df1["1stFlrSF"] + df1["2ndFlrSF"]
df1["TotalSF"] = sum_column
df1

In [ ]:
df1.drop(['Neighborhood', 'BldgType','1stFlrSF','2ndFlrSF','SaleCondition', 'SaleType','Foundation','Electrical','Heating'],axis=1, inplace=True)
df1

In [ ]:
df1['CentralAir'] = df1['CentralAir'].astype('str')
df1.CentralAir.replace(('Y', 'N'), (1, 0), inplace=True)
df1['PavedDrive'] = df1['PavedDrive'].astype('str')
df1.PavedDrive.replace(('Y', 'N','P'), (1, 0,1), inplace=True)
df1.dropna(inplace=True)  
df1.head()          

In [8]:
df1.corr(method='pearson')

,OverallQual,OverallCond,YearBuilt,CentralAir,FullBath,HalfBath,Fireplaces,GarageCars,PavedDrive,MoSold,YrSold,SalePrice,TotalSF
OverallQual,1.000000,-0.091932,0.572323,0.272038,0.550600,0.273458,0.396765,0.600671,0.214103,0.070815,-0.027347,0.790982,0.600758
OverallCond,-0.091932,1.000000,-0.375983,0.118969,-0.194149,-0.060769,-0.023820,-0.185758,-0.044082,-0.003511,0.043950,-0.077856,-0.082724
YearBuilt,0.572323,-0.375983,1.000000,0.381831,0.468271,0.242656,0.147716,0.537850,0.387961,0.012398,-0.013618,0.522897,0.217805
CentralAir,0.272038,0.118969,0.381831,1.000000,0.109326,0.134637,0.186309,0.233726,0.324957,0.009846,-0.009420,0.251328,0.099120
FullBath,0.550600,-0.194149,0.468271,0.109326,1.000000,0.136381,0.243671,0.469672,0.107849,0.055872,-0.019669,0.560664,0.635296
HalfBath,0.273458,-0.060769,0.242656,0.134637,0.136381,1.000000,0.203649,0.219178,0.104569,-0.009050,-0.010269,0.284108,0.421742
Fireplaces,0.396765,-0.023820,0.147716,0.186309,0.243671,0.203649,1.000000,0.300789,0.151006,0.046357,-0.024096,0.466929,0.467488
GarageCars,0.600671,-0.185758,0.537850,0.233726,0.469672,0.219178,0.300789,1.000000,0.263191,0.040522,-0.039117,0.640409,0.479932
PavedDrive,0.214103,-0.044082,0.387961,0.324957,0.107849,0.104569,0.151006,0.263191,1.000000,0.018938,0.000896,0.212630,0.079058
MoSold,0.070815,-0.003511,0.012398,0.009846,0.055872,-0.009050,0.046357,0.040522,0.018938,1.000000,-0.145721,0.046432,0.052725


In [9]:
df1.drop(['OverallCond', 'CentralAir','HalfBath','PavedDrive','MoSold', 'YrSold'],axis=1, inplace=True)
df1

,OverallQual,YearBuilt,FullBath,Fireplaces,GarageCars,SalePrice,TotalSF
0,7,2003,2,0,2,208500,1710
1,6,1976,2,1,2,181500,1262
2,7,2001,2,1,2,223500,1786
3,7,1915,1,1,3,140000,1717
4,8,2000,2,1,3,250000,2198
...,...,...,...,...,...,...,...
1455,6,1999,2,1,2,175000,1647
1456,6,1978,2,2,2,210000,2073
1457,7,1941,2,2,1,266500,2340
1458,5,1950,1,0,1,142125,1078


In [10]:
Q1 = df1.quantile(0.25)
Q3 = df1.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

OverallQual        2.0
YearBuilt         46.0
FullBath           1.0
Fireplaces         1.0
GarageCars         1.0
SalePrice      84025.0
TotalSF          651.5
dtype: float64


In [11]:
df1_out = df1[~((df1 < (Q1 - 1.5 * IQR)) |(df1 > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df1_out.shape)

(1372, 7)


In [12]:
train_df = df1_out

In [13]:
X = train_df.drop(columns=['SalePrice'])
X.describe()


,OverallQual,YearBuilt,FullBath,Fireplaces,GarageCars,TotalSF
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,5.975219,1970.623907,1.534985,0.575802,1.712099,1449.688047
std,1.255522,29.336623,0.530142,0.621237,0.703891,434.244598
min,2.000000,1885.000000,0.000000,0.000000,0.000000,438.000000
25%,5.000000,1953.000000,1.000000,0.000000,1.000000,1111.750000
50%,6.000000,1972.000000,2.000000,1.000000,2.000000,1429.500000
75%,7.000000,1999.000000,2.000000,1.000000,2.000000,1718.000000
max,10.000000,2009.000000,3.000000,2.000000,3.000000,2730.000000


In [14]:
y = train_df[['SalePrice']]
y.describe()


,SalePrice
count,1372.000000
mean,169605.620991
std,58613.095829
min,34900.000000
25%,129000.000000
50%,158950.000000
75%,202500.000000
max,340000.000000


In [15]:
y1 = np.where(y < 169606, 0, y)
y2 = np.where(y > 169606, 1, y1)
y2

array([[1],
       [1],
       [1],
       ...,
       [1],
       [0],
       [0]])

In [16]:
X, Xt, y, yt = train_test_split(X, y2,
                                test_size=0.20, 
                                random_state=0)

**Machine Learning**

In [18]:
regressor = LinearRegression()
regressor.fit(Xt, yt)
print(regressor.intercept_)
y_pred = regressor.predict(X)

[-8.64019842]


In [19]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred)))

Mean Absolute Error: 0.24552079614451167
Mean Squared Error: 0.09574422124030726
Root Mean Squared Error: 0.3094256311948111


In [20]:
Logreg_setup= LogisticRegression()
Logreg_model=Logreg_setup.fit(Xt,yt)
Logreg_pred=Logreg_setup.predict(X)
Logreg_ascore=metrics.accuracy_score(y, Logreg_pred)
print ("Accuracy Score of Logistic Regression is: %f" %(Logreg_ascore))

Accuracy Score of Logistic Regression is: 0.861440


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [21]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(Xt,yt)
y_pred = clf.predict(X)
print("Accuracy:", sklearn.metrics.accuracy_score(y, y_pred))

Accuracy: 0.853236098450319


In [22]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf = clf.fit(Xt,yt)
y_pred = clf.predict(X)
print("Accuracy:", sklearn.metrics.accuracy_score(y, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.8915223336371924


In [23]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf = clf.fit(Xt,yt)
y_pred = clf.predict(X)
print("Accuracy:", sklearn.metrics.accuracy_score(y, y_pred))

Accuracy: 0.8742023701002735


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=(6)))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, validation_split=0.2, epochs=30,)



Epoch 1/30
28/28 [==============================] - 1s 10ms/step - loss: 0.4287 - accuracy: 0.5713 - val_loss: 0.4455 - val_accuracy: 0.5545
Epoch 2/30
28/28 [==============================] - 0s 3ms/step - loss: 0.4287 - accuracy: 0.5713 - val_loss: 0.4455 - val_accuracy: 0.5545
Epoch 3/30
28/28 [==============================] - 0s 4ms/step - loss: 0.4287 - accuracy: 0.5713 - val_loss: 0.4455 - val_accuracy: 0.5545
Epoch 4/30
28/28 [==============================] - 0s 4ms/step - loss: 0.4287 - accuracy: 0.5713 - val_loss: 0.4455 - val_accuracy: 0.5545
Epoch 5/30
28/28 [==============================] - 0s 3ms/step - loss: 0.4287 - accuracy: 0.5713 - val_loss: 0.4455 - val_accuracy: 0.5545
Epoch 6/30
28/28 [==============================] - 0s 4ms/step - loss: 0.4287 - accuracy: 0.5713 - val_loss: 0.4455 - val_accuracy: 0.5545
Epoch 7/30
28/28 [==============================] - 0s 3ms/step - loss: 0.4287 - accuracy: 0.5713 - val_loss: 0.4455 - val_accuracy: 0.5545
Epoch 8/30
28/28 [=

In [26]:
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

35/35 [==============================] - 0s 1ms/step - loss: 0.4321 - accuracy: 0.5679
Accuracy: 56.79


In [27]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=3)
model.fit(X, y, validation_split=0.1, epochs=15, callbacks=[early_stopping_monitor])

Epoch 1/15
31/31 [==============================] - 0s 4ms/step - loss: 0.4336 - accuracy: 0.5664 - val_loss: 0.4182 - val_accuracy: 0.5818
Epoch 2/15
31/31 [==============================] - 0s 3ms/step - loss: 0.4336 - accuracy: 0.5664 - val_loss: 0.4182 - val_accuracy: 0.5818
Epoch 3/15
31/31 [==============================] - 0s 3ms/step - loss: 0.4336 - accuracy: 0.5664 - val_loss: 0.4182 - val_accuracy: 0.5818
Epoch 4/15
31/31 [==============================] - 0s 4ms/step - loss: 0.4336 - accuracy: 0.5664 - val_loss: 0.4182 - val_accuracy: 0.5818


**Deep Learning**

In [28]:
def init(inp, out):
    return np.random.randn(inp, out) / np.sqrt(inp)

def create_architecture(input_layer, first_layer, 
                        output_layer, random_seed=0):
    np.random.seed(random_seed)
    layers = X.shape[1], 3 , 1
    arch = list(zip(layers[:-1], layers[1:]))
    weights = [init(inp, out) for inp, out in arch]
    return weights
    
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_prime(s):
    return s * (1 -s)



In [29]:
def feed_forward(X, weights):
    a = X.copy()
    out = list()
    for W in weights:
        z = np.dot(a, W)
        a = sigmoid(z)
        out.append(a)
    return out

In [30]:
def accuracy(true_label, predicted):
    correct_preds = np.ravel(predicted)==true_label
    return np.sum(correct_preds) / len(true_label)

In [31]:
def back_propagation(l1, l2, weights, test_y):
    l2_error = y.reshape(-1, 1) - l2
    l2_delta = l2_error * sigmoid_prime(l2)
    l1_error = l2_delta.dot(weights[1].T)
    l1_delta = l1_error * sigmoid_prime(l1)
    return l2_error, l1_delta, l2_delta

def update_weights(X, l1, l1_delta, l2_delta, weights, alpha=1.0):
    weights[1] = weights[1] + (alpha * l1.T.dot(l2_delta))
    weights[0] = weights[0] + (alpha * X.T.dot(l1_delta))
    return weights

In [32]:
def predict(X, weights):
    _, l2 = feed_forward(X, weights)
    preds = np.ravel((l2 > 0.5).astype(int))
    return preds

In [33]:
weights = create_architecture(X, 3, 1)

In [34]:
for j in range(30000 + 1):

    l1, l2 = feed_forward(X, weights)
    l2_error, l1_delta, l2_delta = back_propagation(l1, l2, weights, y)
    weights = update_weights(X, l1, l1_delta, l2_delta, weights, alpha=0.05)
    if (j % 5000) == 0:
        train_error = np.mean(np.abs(l2_error))
        print('Epoch {:5}'.format(j), end=' - ')
        print('error: {:0.4f}'.format(train_error), end= ' - ')

        train_accuracy = accuracy(true_label=yt, predicted=(l2 > 1))
        test_preds = predict(X, weights)
        test_accuracy = accuracy(true_label=y, predicted=test_preds)
        
        print('acc: train {:0.3f}'.format(train_accuracy), end= ' | ')
        print('test {:0.3f}'.format(test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


Epoch     0 - error: 0.4895 - acc: train 594.375 | test 623.000
Epoch  5000 - error: 0.4804 - acc: train 594.375 | test 474.000
Epoch 10000 - error: 0.5020 - acc: train 594.375 | test 623.000
Epoch 15000 - error: 0.4328 - acc: train 594.375 | test 623.000
Epoch 20000 - error: 0.4329 - acc: train 594.375 | test 623.000
Epoch 25000 - error: 0.4467 - acc: train 594.375 | test 474.000
Epoch 30000 - error: 0.5518 - acc: train 594.375 | test 623.000
